<a href="https://colab.research.google.com/github/Ryu4824/code-states/blob/main/n212_discussion_6%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **N212 Generalization**

## 오늘의 목표
- 데이터를 학습/검증/테스트셋으로 분리할 수 있습니다.
- 다항 선형회귀 모델에서 최적의 차수를 찾을 수 있습니다.
- 다중 선형회귀 모델과 다항 선형회귀 모델의 결과를 비교할 수 있습니다.
- 테스트셋을 활용해 모델의 일반화 성능을 평가할 수 있습니다.

## **개념 Topic**
> 오늘은 일반화에 대해서 배웠습니다.

- 다음은 일반화 성능을 높이기 위한 몇 가지 방법입니다.
  - 아래 방법은 어떤 문제(과적합/과소적합)를 해결할 수 있는지 설명해 보세요.
  - 왜 그러한지 분산과 편향의 관점에서 설명해 보세요.
    ```markdown
    1. 학습 데이터를 추가한다.
    2. 특성의 수를 줄인다.
    3. 특성의 수를 늘린다.
    4. 차수가 높은 특성을 늘린다.
    ```
    - 2. 특성의 수를 줄인다. → 집의 크기, 방의 개수, 연식 등의 feature 수를 줄이는 것
    - 4. 차수가 높은 특성을 늘린다. → 다항회귀에서 차수를 높이면 차수가 높은 특성이 추가되는 것 

\begin{align}
y = \beta + \beta_1x + \beta_2x^2 + \beta_3x^3 + \cdots
\end{align}

- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

In [1]:
# 1. 학습 데이터(row data)를 추가하면, 과소적합을 해결 할 수 있을 것 같다.
# 학습데이터가 너무 없으면 과적합이 생길 수 있다.
# 편향이 커지면 과소적합이 생기므로 학습데이터를 추가하면 편향을 줄일 수 있다.
# 학습 데이터에 특성이 많을 때 row data가 적으면 

# 모델이 단순하면 과소적합 -> 

# 2. 특성의 수를 줄이면 과적합을 해결 할 수 있을 것 같다.
# 특성이 많으면 모델이 복잡해 지고 과적합이 될 가능성이 있으므로 특성의 수를 줄이면 분산이 작아지므로 과적합을 해결 할 수 있다.

# 3. 특성의 수를 늘리면 과소적합을 해결 할 수 있다.
# 특성의 수를 늘리면 분산이 커지고 과소적합을 해결 할 수 있다.

# 4. 차수가 높은 특성을 늘리면 과소적합을 해결 할 수 있다.
# 

In [2]:
# 모델이 복잡해질수록 편향이 줄어들고 분산이 높아진다.
# (“학습데이터가 작을수록”,특성이 늘어날수록, 차원이 높아질수록)
# 단순해질수록 편향이 높아지고 분산이 줄어든다.
# (“학습데이터가 클수록”, 특성이 줄어들수록, 차원이 작을수록)

# 그러므로 모델의 “적절한” 복잡도를 찾는것이 중요.

## **코딩 Topic**

### **Part.1 : 데이터 준비**

지난 노트에서도 사용했던 인도의 한 도시인 [Begaluru의 집값 데이터](https://www.kaggle.com/datasets/amitabhajoy/bengaluru-house-price-data)를 사용해서 집값을 예측하는 회귀 문제를 풀어보겠습니다.

> **Data Description**

- Area_type : Description of the area
- Availability : When it can be possessed or when it is ready
- Location : Where it is located in Bengaluru
- Size : BHK or Bedrooms
- Society : To which society it belongs
- Total_sqft : Size of the property in sq.ft
- Bath : No. of Bathrooms
- Balcony : No. of the Balcony
- Price : Value of the property in lakhs (Indian Rupee - ₹)


#### **1-1. 데이터셋 로딩 및 가공**
- 지난 노트에서 사용한 데이터를 불러옵니다.
- 지난 디스커션과 같은 방법으로 4개의 특성 선택, 결측치 및 중복값 처리, `total_sqft` 변수의 타입 변환을 수행해보세요.

In [3]:
import re
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [4]:
data = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/renewal/mldl/House_Data.csv')
display(data.head())

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


#### **1-2. 데이터 형태 확인**
- 전처리 후, 지난 노트와 데이터의 shape이 같은지 확인해보세요.
- 특성들의 type을 확인해보세요.
  - Hint) **dtypes**

In [5]:
df = data[['total_sqft', 'bath', 'balcony', 'price']].copy()

In [6]:
df.dropna(axis=0, inplace=True)

In [7]:
df.drop_duplicates(keep='first', inplace=True)

In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
df.shape

(10933, 4)

In [10]:
df.head(5)

,total_sqft,bath,balcony,price
0,1056,2.0,1.0,39.07
1,2600,5.0,3.0,120.00
2,1440,2.0,3.0,62.00
3,1521,3.0,1.0,95.00
4,1200,2.0,1.0,51.00


In [11]:
# 수치로 변환되지 않는 데이터 index 추출
idx = []
for i in range(len(df.total_sqft)):
  try:
    float(df.total_sqft[i])
  except:
    idx.append(i) # 수치형으로 변환되지 않은 데이터들의 index 저장. ex) 2100 - 2850 형식처럼 수치형으로 바꿀 수 없는 데이터 존재.

In [12]:
len(idx)

185

In [13]:
df.iloc[idx[:10]] # 이와 같이 수치형으로 바꾸기 애매한 데이터들 확인.

,total_sqft,bath,balcony,price
27,2100 - 2850,4.0,0.0,186.000
114,3067 - 8156,4.0,0.0,477.000
129,1042 - 1105,2.0,0.0,54.005
152,1145 - 1340,2.0,0.0,43.490
175,1015 - 1540,2.0,0.0,56.800
386,34.46Sq. Meter,1.0,0.0,18.500
515,1195 - 1440,2.0,0.0,63.770
618,1120 - 1145,2.0,0.0,48.130
628,3090 - 5002,4.0,0.0,445.000
724,1160 - 1195,2.0,0.0,59.935


In [14]:
# 수치형으로 변환되지 않는 값의 index에 해당하는 샘플 제거
df = df.drop(idx, axis=0)
df.reset_index(drop=True, inplace=True)
df.shape

(10748, 4)

In [15]:
# total_sqft를 float형으로 변환
df['total_sqft'] = df['total_sqft'].astype('float')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10748 entries, 0 to 10747
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_sqft  10748 non-null  float64
 1   bath        10748 non-null  float64
 2   balcony     10748 non-null  float64
 3   price       10748 non-null  float64
dtypes: float64(4)
memory usage: 336.0 KB


### **Part.2 : 모델링**

#### **2-1. 데이터 분리 및 기준 모델 설정**
- 타겟은 `price`, 다른 특성들은 입력 특성으로 사용하세요.
- 데이터를 학습/테스트셋으로 분리해보세요.
  ```python
  from sklearn.model_selection import train_test_split
  ```
  - `random_state` 값으로 고정
  - `test size = 0.2`
- 분리된 학습 데이터를 학습/검증셋으로 다시 분리하세요.
  - `random_state` 값으로 고정
  - `test size = 0.2`
- 분리 후 X_train, y_train, X_val, y_val, X_test, y_test의 데이터 shape은 각각 어떻게 되나요?
- 학습 데이터의 타겟인 y_train의 평균을 사용하여 기준 모델을 설정하세요.
  - 기준 모델의 MAE는 어떻게 되나요?

In [26]:
target = 'price'
X = df.drop(target,axis=1)
y = df[target]

X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)
X_train, X_val, y_train,y_val = train_test_split(X_train,y_train,test_size=0.2, random_state=42)

print('학습용:',X_train.shape,y_train.shape)
print('검증용:',X_val.shape,y_val.shape)
print('평가용:',X_test.shape,y_test.shape)

학습용: (6878, 3) (6878,)
검증용: (1720, 3) (1720,)
평가용: (2150, 3) (2150,)


#### **2-2. 다중 선형회귀 모델**
- 모든 입력 특성을 사용해서 다중 선형회귀 모델을 만들고 학습과 검증 데이터셋에서의 MAE와 $R^2$ score를 확인해보세요.
  - MAE: `mean_absolute_error(실제값, 예측값)`
  - $R^2$: `선형회귀모델.score(입력 특성, 타겟)`
- 과적합/과소적합이 발생했나요? 발생했다면 어떻게 해결할 수 있을까요?
  - 편향과 분산의 관점에서 해결 방법을 논의해보세요.

In [19]:
def eval_models(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, rmse, mae, r2

In [20]:
multiple_ols = LinearRegression()
multiple_ols.fit(X_train, y_train)

LinearRegression()

In [35]:
y_train_pred = multiple_ols.predict(X_train)
y_val_pred = multiple_ols.predict(X_val)

In [37]:
eval_models(y_train, y_train_pred)

(12400.485914155599, 111.35746905419322, 46.43913573324582, 0.3405161065179958)

In [38]:
eval_models(y_val, y_val_pred)

(8922.335390045559, 94.45811447432962, 45.29814063546828, 0.4772697242059234)

In [24]:
# 충분한 학습을 하지않아 과소적합이 발생했습니다.

#### **2-3. 다항 선형회귀 모델**
- 모든 입력 특성을 사용해서 다항 선형회귀 모델을 만들어보세요.
  - 차수는 주어진 리스트 내에 있는 차수를 모두 사용해보세요.
  - 각 차수에 해당하는 다항 선형회귀 모델의 학습과 검증 데이터셋에서의 MAE, $R^2$ score를 확인해보세요.
- 차수가 몇일 때 과적합이 가장 심했나요?
  - 높은 차수는 낮은 차수에 비해 학습과 검증 데이터셋에서의 성능 차이가 **항상** 클까요?
- 다중 선형회귀와 다항 선형회귀 중 성능이 더 좋은 모델은 어떤 모델인가요?
  - 더 좋은 모델은 안 좋은 모델에 비해 검증 데이터에서의 MAE가 얼마나 더 낮나요?

In [39]:
from IPython.display import display, HTML
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), 
                         LinearRegression(**kwargs))

In [25]:
polynomial_degrees = [1, 2, 3, 4, 5]

In [47]:
for degree in polynomial_degrees:
    model = PolynomialRegression(degree)
    print(f'Degree={degree}')
    
    model.fit(X_train, y_train)
    train_r2 = model.score(X_train, y_train)
    val_r2 = model.score(X_val, y_val)
    test_r2 = model.score(X_test, y_test)

    display(HTML(f'train R2 {train_r2:.3f}'))
    display(HTML(f'val R2 {val_r2:.3f}'))
    display(HTML(f'test R2 {test_r2:.3f}'))
    display(HTML('<hr/>'))


y_train_pred = multiple_ols.predict(X_train)
y_val_pred = multiple_ols.predict(X_val)

mae_t = mean_absolute_error(y_train,y_train_pred)
mae_v = mean_absolute_error(y_val,y_val_pred)
display(HTML(f'train MAE {mae_t:.3f}'))
display(HTML(f'val MAE {mae_v:.3f}'))

Degree=1


Degree=2


Degree=3


Degree=4


Degree=5


2 차수가 가장 높은 성능을 보여준다. 그리고 4일때 가장 과적합이 일어났고, 다항 선형괴귀가 성능이 더 좋았다

#### **2-4. 기준 모델과 성능 비교**
- 모델링 결과 성능이 가장 좋은 모델을 최종 모델로 설정해보세요.
  - 다중 선형회귀 모델 OR 다항 선형회귀 모델(차수가 몇일 때)
- 최종 모델은 기준 모델보다 성능이 좋나요?
  - 성능을 평가하는 용도로 테스트셋의 MAE를 활용해보세요.
  - 최종 모델과 기준 모델의 성능을 비교해보세요.


In [48]:
model = PolynomialRegression(2)
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
mae_test = mean_absolute_error(y_test,y_test_pred)

test_r2 = model.score(X_test, y_test)


In [49]:
#기준 모델
baseline = [y_train.mean()] * len(y_train)
base_mae = mean_absolute_error(y_train,baseline)
base_score = r2_score(y_train, baseline)

result = pd.DataFrame(index=['mae', 'r2'], columns=['Base','Test'])
result['Base'] = base_mae,base_score
result['Test'] = mae_test,test_r2
result

,Base,Test
mae,71.169937,46.154565
r2,0.000000,0.531569


#### **Optional) 과적합/과소적합을 해소하기 위한 방법을 코드로 구현해보세요.** 
  - 새로운 특성 만들기, 다항회귀 차수 조정 등등
  - 과적합/과소적합이 해소 되었나요?

### **Conclusion**
> 오늘 Topic을 수행한 결과를 바탕으로, 다음 사항에 대해 답해 주세요.

- 데이터를 학습, 검증, 테스트셋으로 분리하는 이유가 무엇일까요?
- 다중 선형회귀와 다항 선형회귀 모델 중 더 좋은 모델은 무엇인가요?
- 다항 선형회귀 모델의 최적의 차수는 몇인가요?
- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 